In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy import stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import gc
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.externals import joblib
from sklearn.svm import SVC
import sklearn.metrics as metrics
import xgboost as xgb
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

path = '../data/Debt issuing company 2018 report/'

In [12]:
def drop_out(frame,col,model='Confidence interval',t_alpha=0.95,alpha=2,IQR_rate=1.5,quantile=0.95):
    '''modle: 'gauss','box','quantile' '''
    
    if model == 'Confidence interval':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        interval_ = stats.t.interval(t_alpha,frame[col].count()-1,u_,v_)
        cond_ = (frame[col]<interval_[1])&(frame[col]>interval_[0])
    
    elif model == 'gauss':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        cond_ = np.abs((frame[col]-u_)/v_) < alpha
    
    elif model == 'box':
        q1 = frame[col].quantile(0.25)
        q3 = frame[col].quantile(0.75)
        IQR = (q3-q1)*IQR_rate
        q1 -= IQR ; q3 += IQR
        cond_ = (frame[col]<q3)&(frame[col]>q1)
    
    elif model == 'quantile':
        top_ = frame[col].quantile(quantile)
        bottom_ = frame[col].quantile(1-quantile)
        cond_ = (frame[col]<top_)&(frame[col]>bottom_)
    
    else:
        print('please try again')
        return frame
    
    index_ = np.where(frame[col]!=frame[col],True,
                                                  np.where(cond_,True,False))
    frame = frame.loc[index_,:]
    return frame

In [67]:
start_year = 2013
end_year = 2018
data_a = DataFrame()
for i in range(start_year,end_year+1):
    if i % 3 ==0:
        print('is concating {} {}/{}'.format(i,i-start_year+1,end_year+1-start_year))
    try:
        path_a = path+f'y/{i}y.xlsx'
        data_a_ = pd.read_excel(path_a)[:-2]
        data_a_.drop(['是否经过审计','审计意见']+[i for i in data_a_.columns if i.find('E')!=-1],axis=1,inplace=True)
    #     data_a_.loc[:,['主营业务收入(亿元)','主营业务利润(亿元)','净利润(亿元)']] = \
    #     data_a_.loc[:,['主营业务收入(亿元)','主营业务利润(亿元)','净利润(亿元)']].apply(lambda x:x/data_a_['报告期'].dt.month)
        data_a = pd.concat([data_a,data_a_])
    except:
        print(f'no {i}')
        pass
del data_a_
gc.collect()
print('finish concat data_y')

print(np.array(list(data_a.isnull().sum(0))))
# data_a.dropna(thresh=data_a.shape[1]-6,inplace=True)

is concating 2013 1/6
is concating 2016 4/6
no 2018
finish concat data_y
[   0    0  177 2209  225  418 2782 2782  620  831  741  357  705  568
  799 1400 2798  795 1771 2615 3442  735 2788 2789 2727 2214 6719]


In [68]:
for k in range(1):
    col_ = np.random.choice(data_a.columns[2:],len(data_a.columns[2:]),replace=False,)
    for j in col_:
        data_a = drop_out(data_a,j,model='gauss',alpha=3)

In [106]:
for i in range(data_a.shape[1]-2):
    ax = plt.subplot(5,5,i+1)
    ax.scatter(range(data_a.shape[0]),data_a.iloc[:,i+2].sort_values(),s=3)
    plt.title(data_a.columns[i+2])
plt.suptitle('散点趋势图')

Text(0.5,0.98,'散点趋势图')

In [69]:
all_com = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0827.xlsx',sheet_name='产业类企业')
all_com = all_com[['名称', '最新评级', '企业性质', '是否上市','一级分类', '二级分类']]

In [70]:
a = data_a.merge(all_com[['名称','企业性质', '是否上市','一级分类','二级分类']],on='名称').copy()

In [71]:
a = a.loc[a['报告期'].dt.month == 12,:]

In [72]:
excelname = '去异小类年度平均数'
Exceleidtor = pd.ExcelWriter(f'../{excelname}.xlsx',)

In [73]:
a[list(a.columns[1:-2])+['二级分类']].groupby(['二级分类','报告期']).mean().to_excel(Exceleidtor,'origin_mean')
a[list(a.columns[1:-2])+['二级分类']].groupby(['二级分类','报告期']).median().to_excel(Exceleidtor,'origin_median')

In [74]:
if __name__ == '__main__':
    def diy_ss(frame,quantile=0.2):
        weight_dict = {}
        for i in frame.columns:
            if frame[i].min() >= 0:
                frame[i] = np.log1p(frame[i])
                weight_dict[i] = 'log1p'
            else :
                mean_ = frame[i][(frame[i]>frame[i].quantile(quantile)) & (frame[i]<frame[i].quantile(1-quantile))].mean()
                std_ = frame[i][(frame[i]>frame[i].quantile(quantile)) & (frame[i]<frame[i].quantile(1-quantile))].std()
                weight_dict[i] = [mean_,std_]
                frame[i] = (frame[i]-mean_)/std_
        return frame,weight_dict
    def fillna_(frame,group_col,quantile=0.2,n_epoch=3):

        def return_index(aa,bb):
            j=0
            cc = []
            for i in range(len(aa)):
                if aa[i] == False :
                    cc.append(aa[i])
                else:
                    cc.append(bb[j])
                    j += 1
            return cc

        frame_col = frame.columns

        frame.iloc[:,:-7],weight_dict = diy_ss(frame.iloc[:,:-7],quantile)

        for l,comp in enumerate(set(group_col)):

            index_y = list(group_col == comp)
            full_col = []
            loss_col = {}

            for col in frame.columns:
                if frame.loc[index_y,col].isnull().sum() == 0:
                    full_col.append(col)
                else:
                    loss_col[col] = frame.loc[index_y,col].isnull().sum()

            loss_col = sorted(loss_col.items(),key=lambda x:x[1])
            loss_col = [i[0] for i in loss_col]

            index_dict = {}
            if len(full_col) == 0:
                index_dict[loss_col[0]] = frame.loc[index_y,loss_col[0]].isnull()
                index_dict[loss_col[0]].fillna(index_dict[loss_col[0]].median(),inplace=True)
                full_col.append(loss_col[0])
                loss_col = loss_col[1:]

            for epoch in range(n_epoch):

                if epoch == 0:
                    for _,col in enumerate(loss_col):
                        if np.random.rand()>0.75:
                            print(comp,f'{l}/{len(set(group_col))}',col,f'{_}/{len(loss_col)}')
                        index_l = list(frame.loc[index_y,col].isnull())
                        index_f = list(frame.loc[index_y,col].notnull())
                        index_l_ = return_index(index_y,index_l)
                        index_f_ = return_index(index_y,index_f)
                        index_dict[col] = (index_l_,index_f_)
                        rfr = RandomForestRegressor(n_estimators=10,n_jobs=-1,max_features=0.9)
                        rfr.fit(frame.loc[index_f_,full_col],frame.loc[index_f_,col])
                        pre = rfr.predict(frame.loc[index_l_,full_col])
                        frame.loc[index_l_,col] = pre
                        full_col.append(col)

                else:
                    for col in index_dict:
                        index_l_ = index_dict[col][0]
                        index_f_ = index_dict[col][1]
                        rfr = RandomForestRegressor(n_estimators=20,n_jobs=-1,max_features=0.6)
                        rfr.fit(frame.loc[index_f_,full_col],frame.loc[index_f_,col])
                        pre = rfr.predict(frame.loc[index_l_,full_col])
                        frame.loc[index_l_,col] = pre

        gc.collect()
        return frame[frame_col],weight_dict

In [75]:
def one_hot_str(frame,col,replace=True):
    if replace:
        a_ = frame.pop(col)
    else :
        a_ = frame[col]
    a_.fillna('miss',inplace=True)
    a_ = pd.get_dummies(a_,prefix=a_.name,)
    frame = pd.concat([frame,a_],axis=1)
    del a_
    return frame

In [76]:
a = one_hot_str(a,'报告期',replace=False)
a = one_hot_str(a,'是否上市',)

In [77]:
for i in range(a.shape[1]):
    sum_ = a.iloc[:,i].isnull().sum()
    print(sum_/a.shape[0],a.columns[i],)

0.0 名称
0.0 报告期
0.01180549822504747 总资产(亿元)
0.04251630479649963 货币资产(亿元)
0.014860067695863947 净资产(亿元)
0.028564352348716255 总债务(亿元)
0.043424420044580206 带息债务(亿元)
0.043424420044580206 净债务(亿元)
0.03805828448774044 经营活动现金流(亿元)
0.04301164038636176 投资活动现金流(亿元)
0.04036985057376372 筹资活动现金流(亿元)
0.02063898291092215 主营业务收入(亿元)
0.041525633616775366 主营业务利润(亿元)
0.03698505737637249 净利润(亿元)
0.04466275901923553 主营业务利润率(%)
0.05770659621893833 主营业务收入增长率(%)
0.043424420044580206 总资产报酬率(%)
0.04367208783951127 净资产回报率(%)
0.043754643771154955 流动比率
0.043754643771154955 速动比率
0.06472385040865186 存货周转率
0.04276397259143069 资产负债率
0.043424420044580206 短期债务/总债务
0.04358953190786758 带息债务/总投入资本
0.04160818954841906 货币资金/短期债务
0.042846528523074384 货币资金/总债务
0.13159415504003963 获息倍数
8.25559316436886e-05 企业性质
0.0 一级分类
0.0 二级分类
0.0 报告期_2013-12-31 00:00:00
0.0 报告期_2014-12-31 00:00:00
0.0 报告期_2015-12-31 00:00:00
0.0 报告期_2016-12-31 00:00:00
0.0 报告期_2017-12-31 00:00:00
0.0 是否上市_否
0.0 是否上市_是


In [78]:
a.loc[:,a.columns.drop(['名称','报告期','一级分类', '二级分类','企业性质'])],_ = \
fillna_(a.loc[:,a.columns.drop(['名称','报告期','一级分类', '二级分类','企业性质'])],a['一级分类'],n_epoch=4)

房地产开发 0/18 货币资金/短期债务 6/25
房地产开发 0/18 主营业务利润(亿元) 9/25
房地产开发 0/18 带息债务(亿元) 12/25
房地产开发 0/18 主营业务收入增长率(%) 23/25
房地产开发 0/18 获息倍数 24/25
新能源 2/18 总债务(亿元) 2/25
新能源 2/18 主营业务收入增长率(%) 13/25
新能源 2/18 资产负债率 18/25
新能源 2/18 货币资金/总债务 22/25
新能源 2/18 存货周转率 23/25
保障类消费 3/18 带息债务(亿元) 1/21
保障类消费 3/18 主营业务利润率(%) 8/21
保障类消费 3/18 总资产报酬率(%) 9/21
保障类消费 3/18 速动比率 12/21
保障类消费 3/18 带息债务/总投入资本 15/21
投资贸易 4/18 净利润(亿元) 4/25
投资贸易 4/18 货币资金/短期债务 5/25
投资贸易 4/18 经营活动现金流(亿元) 7/25
投资贸易 4/18 货币资产(亿元) 9/25
投资贸易 4/18 带息债务(亿元) 10/25
投资贸易 4/18 净债务(亿元) 11/25
投资贸易 4/18 主营业务利润率(%) 20/25
新材料 5/18 速动比率 2/5
新材料 5/18 货币资金/总债务 3/5
物流快递 6/18 总债务(亿元) 0/23
物流快递 6/18 净利润(亿元) 4/23
物流快递 6/18 流动比率 12/23
物流快递 6/18 存货周转率 21/23
物流快递 6/18 获息倍数 22/23
商务服务业 7/18 主营业务收入增长率(%) 2/13
商务服务业 7/18 带息债务(亿元) 3/13
商务服务业 7/18 速动比率 7/13
商务服务业 7/18 短期债务/总债务 8/13
商务服务业 7/18 货币资金/短期债务 10/13
节能环保 8/18 总债务(亿元) 4/25
节能环保 8/18 经营活动现金流(亿元) 7/25
节能环保 8/18 投资活动现金流(亿元) 8/25
节能环保 8/18 主营业务利润(亿元) 9/25
节能环保 8/18 总资产报酬率(%) 12/25
节能环保 8/18 速动比率 15/25
节能环保 8/18 短期债务/总债务 18/2

In [79]:
for i in range(a.shape[1]):
    sum_ = a.iloc[:,i].isnull().sum()
    print(sum_/a.shape[0],a.columns[i],)

0.0 名称
0.0 报告期
0.0 总资产(亿元)
0.0 货币资产(亿元)
0.0 净资产(亿元)
0.0 总债务(亿元)
0.0 带息债务(亿元)
0.0 净债务(亿元)
0.0 经营活动现金流(亿元)
0.0 投资活动现金流(亿元)
0.0 筹资活动现金流(亿元)
0.0 主营业务收入(亿元)
0.0 主营业务利润(亿元)
0.0 净利润(亿元)
0.0 主营业务利润率(%)
0.0 主营业务收入增长率(%)
0.0 总资产报酬率(%)
0.0 净资产回报率(%)
0.0 流动比率
0.0 速动比率
0.0 存货周转率
0.0 资产负债率
0.0 短期债务/总债务
0.0 带息债务/总投入资本
0.0 货币资金/短期债务
0.0 货币资金/总债务
0.0 获息倍数
8.25559316436886e-05 企业性质
0.0 一级分类
0.0 二级分类
0.0 报告期_2013-12-31 00:00:00
0.0 报告期_2014-12-31 00:00:00
0.0 报告期_2015-12-31 00:00:00
0.0 报告期_2016-12-31 00:00:00
0.0 报告期_2017-12-31 00:00:00
0.0 是否上市_否
0.0 是否上市_是


In [80]:
a = pd.concat([a.loc[:,:'获息倍数'],a.loc[:,'二级分类']],axis=1)

In [81]:
def rechange(dataframe,col,weight):
    if weight[col] == 'log1p':
        return np.exp(dataframe[col])-1
    else:
        return dataframe[col]*weight[col][1]+weight[col][0]

for i in a.columns:
    if i in _:
        a.loc[:,i] = rechange(a,i,_)

In [82]:
a.groupby(['二级分类','报告期']).mean().to_excel(Exceleidtor,'filled_mean')
a.groupby(['二级分类','报告期']).median().to_excel(Exceleidtor,'filled_median')

In [83]:
Exceleidtor = pd.ExcelWriter(f'../{excelname}.xlsx',)